# 第6回

## 前回の課題の補足

一度パターンデータベースを構築したら、保存して再利用したい。
Pythonの場合には pickle を使えば簡単に実現できる。
C++ でも serialise, deserialise で同様のことが可能。
以下の例の様にハッシュテーブル（だけでなくほぼ任意のデータ構造）をファイルに保存できる。
保存したファイルを読み出せばそのままの状態で使える。
以下を実行すると、実際に 'dict.pickle' というファイルが作られる。

In [ ]:
'''
pickle で hash table を保存/読込
'''

import pickle

dict = {}
dict['a'] = 1
dict['b'] = 2

print(dict)

# dict をファイルに保存
with open('dict.pickle', 'wb') as handle:
    pickle.dump(dict, handle)

# dict をファイルから読み出し
with open('dict.pickle', 'rb') as handle:
    dict2 = pickle.load(handle)

print(dict2)

## 敵のいる場合の探索

最短経路を探索しようとしている人や、パズルやゲームをプレイしているアルゴリズムなど、
何らかの行動をする存在をまとめて **エージェント (agent)** と呼ぶ。
今までに取り上げた問題は全て、エージェントが一人しかいない single agent の問題だった。

エージェントが複数存在する multi agent の問題は当然ながら single agent の場合よりも複雑になる。
特にエージェント間の利害が対立する場合は最適な行動というものの定義が既に難しい。
一般の multi agent 問題は**ゲーム理論**の扱うテーマであり、本講義では深入りしない。
しかしエージェントが2人までなら、特定の場合には厳密な最適解を探索によって発見することができる。

### 二人ゼロ和完全情報ゲーム

それが **二人ゼロ和ゲーム完全情報 (two-player zero-sum perfect-information game)** と言われる場合である。
より正確には「有限」と「確定」を付けて二人ゼロ和有限確定完全情報ゲームと呼ぶこともある。
以下の条件を満たすゲームのことを言う。

- 2人のプレイヤーがプレイする (ここではmaxプレイヤーとminプレイヤーと呼ぶ)
- ゼロ和 (zero-sum): maxプレイヤーが勝てば、minプレイヤーはその分だけ負ける
- 完全情報: 隠された情報はなく、全てが両方のプレイヤーに見えている
- 有限: 有限時間で終わる
- 確定: サイコロなどの確率的な要素がない

チェス、囲碁、将棋、オセロなどが代表例である。
以下は三目並べ (tic-tac-toe) のゲーム木である。
終局を示す節点には、先手 (maxプレイヤー) が勝ちなら +1、
後手 (minプレイヤー) が勝ちなら -1、引き分けなら 0 のスコアがあるとしている。
先手はスコアを最大化しようとし、後手はスコアを最小化しようとするので
それぞれ max プレイヤー、minプレイヤーと呼ぶ。

<img src="tictactoe.png" width=600>

このようなゲーム木も初期局面をスタート節点とするグラフであり、
直感的にはこのグラフを探索すれば最善手を求めることができそうに思える。
しかし今までに扱ってきたA\*探索などのアルゴリズムではそれはできない。
この場合には相手のプレイヤーの存在を考慮したアルゴリズムが必要となる。

## MiniMax探索 (MiniMax search)

まず、以下の仮想的なゲーム木を見てみよう。
初期局面から初めて3手で終わるゲームであり、末端の節点にスコアが示されている。
max nodeから出る辺はmaxプレイヤーの着手を示す。min node から出る辺は min プレイヤーの着手である。
この場合、双方のプレイヤーが最善を尽くしたら最終的なスコアはどうなるだろうか。

<img src="minimax.gif" width=600>

この図では左側の辺から順番にたどって深さ優先探索を行ってスコアを求めている。
1. max プレイヤーはスコアを最大化したい。
    末端まで探索し、50と24の大きい方を選ぶ。その右側では70を選ぶ。
1. min プレイヤーはスコアを最小化したい。
    50と70が判明した時点でその小さい方、50を選ぶ。
1. 以下繰り返す

ここで示したものが minimax 探索、min-max 探索などと呼ばれるアルゴリズムの動作である。
補足だが、二人ゲームの場合には、探索の開始節点を **根節点 (root)**
辺を **着手 (move)** と呼ぶことが多い。
また、終局した状態 (どちらかの勝ちか引き分け) の節点を **終端節点 (terminal node)** あるいは
単に終端 （terminal) と言う。

以下にminimax探索の疑似コードを示す。
MaxSearch, MinSearchを交互に呼び出す再帰呼び出しでの例である。
MaxSearch は先手プレイヤーの手番での探索で、最大の値を得ようとする。
MinSearch は逆に最小の値を得ようとする。

```
fun MinimaxSearch(game, state) {
  player = Max_player
  value, move = MaxSearch(game, state)
}

fun MaxSearch(game, state) {
  if (state is terminal) { return state.score }
  best_val = -inf
  best_move = dummy
  for each move at state {
    next_state = game.make_move(state, move)
    val = MinSearch(game, next_state)
    if (val > best_val) {
      best_val = val
      best_move = move
    }
  }
  return (best_val, best_move)
}

fun MinSearch(game, state) {
  if (state is terminal) { return state.score }
  best_val = +inf
  best_move = dummy
  for each move at state {
    next_state = game.make_move(state, move)
    val = MaxSearch(game, next_state)
    if (val < best_val) {
      best_val = val
      best_move = move
    }
  }
  return (best_val, best_move)
}
```

細かいところだが、スコアを返すときにプラスマイナスを反転すると
MaxSearchとMinSearchを同じコードで実装することができ、
それを NegaMax と呼ぶことがある。
実際に MiniMax 探索を実装する際には NegaMax にするのが普通である。
以下がNegaMaxの疑似コードである。
注意点としては、NegaMaxを再帰呼び出しするときにその正負を反転すること、
スコアを返すときにその時点の player の視点からのスコアを返すことの2点である。

```
fun NegaMaxSearch(game, state) {
  player = Max_player
  value, move = NegaMax(game, state, player)
}

fun NegaMax(game, player) {
  if (state is terminal) { return evaluate(state, player) }
  best_val = -inf
  best_move = dummy
  for each move at state {
    next_state = game.make_move(state, move)
    next_player = flip(player)
    val = -NegaMax(game, next_state, next_player)
    if (val > best_val) {
      best_val = val
      best_move = move
    }
  }
  return (best_val, best_move)
}
```

### Alpha-Beta 枝刈り (Alpha-Beta探索)

先ほど示した MiniMax 探索は色々と無駄がある。
簡単に分かる点としては、探索中に val に +1 が返ってきたら、その先を探索しなくても勝ちは決まっているので省略して良い。
しかし探索の動作をよく観察すると、探索しなくて良い場合がさらに存在することが分かる。
詳しくは以下の図で説明するが、一言で言えば良すぎる手、悪すぎる手を枝刈りできるということである。
良すぎる手を刈る alpha 枝刈り、悪すぎる手を刈る beta 枝刈りを行うので
これを Alpha-Beta 枝刈り (Alpha-Beta pruning) と言う。
さらにこれを用いる Minimax探索を **AlphaBeta探索** と言う。

<img src="alphabeta.gif" width=600>

評価値が**window**の中にあるなら良すぎも悪すぎもしない。
逆にwindowの外にある値が返ったら即座に枝刈りされる。

<img src="alphabetawindow.png" width=600>

以下に疑似コードを示す。
関数NegaAlphaBetaの最後の二つの引数が上記の window を示す。
最初は $ [-inf, +inf] $ である。
探索中に、自分の手番からは最低限 alpha_value 以上を返せることになるので
より高いスコアが返ってきたらそれで alpha_value を更新している。
また、beta_value より高い値が返ってきたらそれは良すぎることを意味するので
それ以上の探索を打ち切り、即座に return する。

```
fun AlphaBetaSearch(game, state) {
  player = Max_player
  value, move = NegaAlphaBeta(game, state, -inf, +inf)
}

fun NegaAlphaBeta(game, player, alpha_value, beta_value) {
  if (state is terminal) { return evaluate(state, player) }
  best_val = -inf
  best_move = dummy
  for each move at state {
    next_state = game.make_move(state, move)
    next_player = flip(player)
    val = -NegaAlphaBeta(game, next_state, -beta_value, -alpha_value)
    if (val > best_val) {
      best_val = val
      best_move = move
      alpha_value = Max(alpha_value, val)
    }
    if (val >= beta) { return (val, move) }
  }
  return (best_val, best_move)
}
```

### 評価関数つき Alpha-Beta search (Heuristic Alpha-Beta search)

ここまで例では、ゲームのスコアを実際に末端 (terminal) まで探索して求めていた。
この方法では探索空間の小さいゲームは良いが、ある程度以上大きいゲームは解けない。
そこで、末端まで探索するのではなく、ある程度の深さまで探索してそこで
**評価関数 (evaluation function)** を呼ぶ方法がある。
評価関数は実際のゲームのスコアを近似して予測する何らかの関数である。

ある程度正確な評価関数を作成すれば、深く探索することでかなり強い
プログラムを作成することが可能だ。
チェスなどでは人間がプログラムした (hand-craftedな) 評価関数と Alpha-Beta探索で
世界チャンピオンにコンピュータが勝利している。
将棋などでは機械学習で作られた評価関数を用いてやはりコンピュータが人間を超える強さを獲得している。
評価関数の作成は容易ではないことが多く、興味深いテーマだが
本講義では説明時間が足りないため詳細には触れない。

### その他のテクニック

3目並べのゲーム木を見れば明らかだが、ゲームの探索空間では合流が起きることがある。
上で示した疑似コードは合流に対応していないためこの点も無駄である。
しかしMiniMax探索やAlpha-Beta探索も、ハッシュテーブルを使って同じ節点を再展開することを回避できる。

また評価関数を使った Alpha-Beta探索の場合に特に反復深化も使われる。
浅い探索はすぐに終了するため、深さを徐々に増やしながら探索を行うことで
ある程度の Anytime性を得ることもできる。
制限時間内に着手する必要のある対戦などではこの点も重要だ。

探索の深さについては、有望な手は深く、そうでないところは浅く探索するように制御される。
これもプログラムの強さのためには非常に重要なテクニックである。
人工知能の世界では長らくチェス等が知性の象徴とみなされていたため、
Alpha-Beta探索の技術はかなり発展した。


二人ゼロ和ゲーム
参考:
https://qiita.com/thun-c/items/058743a25c37c87b8aa4


### 第6回課題
- 6-1. 3目並べを Minimax探索または Alpha-Beta 探索で解け。適当な盤面を与えて、勝敗と最善手を正しく表示することを確かめよ。
    以下のサンプルコードを利用しても良い。
- 6-2. (発展課題) connect-four (重力付き4目並べ) を Alpha-Beta 探索で解くプログラムを書き、適当な初期盤面から正しく動作することを確かめよ。  
    https://en.wikipedia.org/wiki/Connect_Four  
    https://ja.wikipedia.org/wiki/%E5%9B%9B%E7%9B%AE%E4%B8%A6%E3%81%B9  

## 講義アンケート回答について

```
講義アンケートは　６月１７日（金）までに回答するようにご指示ください．

※※※※※　アンケートの方法　※※※※※

Moodleの「2022年度授業アンケート(2022 Course evaluation questionnaire)」
https://moodle.s.kyushu-u.ac.jp/course/view.php?id=40030
へアクセスし（SSO-KIDでのログインが必要です），以下の項目のリンクをクリックしてください．

2022年度春学期 大学院システム情報科学府授業アンケート: Questionnaire for ISEE Courses, Spring Quarter, 2022

科目名と担当教員の一覧が表示されますので，回答する科目を選択し，回答してください．

※※※※※※※※※※※※※※※※※※※※
```

参考文献
1. "Artificial Intelligence: A Modern Approach, 4th Global ed.", by Stuart Russell and Peter Norvig  
   http://aima.cs.berkeley.edu/index.html
1. "ヒューリスティック探索入門", 陣内 佑  
   https://jinnaiyuu.github.io/pdf/textbook.pdf

# 課題1

In [82]:
import copy

class TicTacToe:
    def __init__(self):
        '''
        player_id = 0 (X) or 1 (O)
        initial state
        [[' ',' ',' '],
         [' ',' ',' '],
         [' ',' ',' ']]                           
        '''
        self.players = ['X', 'O']
        
    def is_terminal(self, state):
        '''
        return true if state is terminal
        '''
        if self.is_draw(state):
            return True
        elif self.is_win(state,0) or self.is_win(state,1):
            return True
        return False
        
    def is_draw(self, state):
        '''
        state が引き分けかどうか判定する関数を実装せよ
        '''
        if self.is_win(state,0) or self.is_win(state,1):
            return False
        for i in state:
            for j in i:
                if j==' ':
                    return False
        return True

    def is_win(self, state, player_id):
        '''
        state と player から勝ちを判定する関数を実装せよ
        '''
        a=self.players[player_id]
        li=[]
        for i in range(3):
            if state[i]==[a,a,a]:
                return True
            for j in range(3):
                if state[i][j]==a:
                    li.append([i,j])
        pat=[[[i,j] for i in [0,1,2]] for j in [0,1,2]] +[[[0,0],[1,1],[2,2]]]+[[[0,2],[1,1],[2,0]]]
        for i in pat:
            x=0
            for j in i:
                if j in li:
                    x+=1
            if x==3:
                return True
        return False
        
    def make_move(self, state, move, player_id):
        '''
        return next state
        '''
        new_state = copy.deepcopy(state)
        x, y = move
        char = self.players[player_id]
        new_state[x][y] = char
        return new_state

    def NegaMax(self, state, player_id):
        if self.is_draw(state):
            return 0,[]
        elif self.is_win(state, player_id):
            return 1,[]
        elif self.is_win(state, 1 - player_id):
            return -1,[]
        
        best_value = -100
        for i in range(3):
            for j in range(3):
                if state[i][j]==' ':
                    next_state = self.make_move(state, [i,j],player_id)
                    next_player = 1-player_id
                    val, move1= self.NegaMax(next_state, next_player)
                    val=-val
                    if val > best_value:
                        best_value = val
                        best_move = [i,j]
        return best_value,best_move
    
    def Score(self):
        return NegaMax(self.state, self.first_player)
    
game = TicTacToe()
# root_state = [[' ',' ',' '],
#              [' ',' ',' '],
#              [' ',' ',' ']]                           
root_state = [[' ',' ','X'],
              [' ','O',' '],
              ['X',' ','O']]                           

score = game.NegaMax(root_state, 0)
print (score)

(1, [0, 0])


In [80]:
def show(state):
    for i in state:
        print(i)
    print('--------------------')

    
game = TicTacToe()
root_state = [[' ',' ',' '],
             [' ',' ',' '],
             [' ',' ',' ']]                           
# root_state = [[' ',' ','X'],
#               [' ','O',' '],
#               ['X',' ','O']]
player=0
while not game.is_terminal(root_state):
    show(root_state)
    score = game.NegaMax(root_state, player)
    root_state[score[1][0]][score[1][1]]=game.players[player]
    player=1-player
show(root_state)

[' ', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']
--------------------
['X', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']
--------------------
['X', ' ', ' ']
[' ', 'O', ' ']
[' ', ' ', ' ']
--------------------
['X', 'X', ' ']
[' ', 'O', ' ']
[' ', ' ', ' ']
--------------------
['X', 'X', 'O']
[' ', 'O', ' ']
[' ', ' ', ' ']
--------------------
['X', 'X', 'O']
[' ', 'O', ' ']
['X', ' ', ' ']
--------------------
['X', 'X', 'O']
['O', 'O', ' ']
['X', ' ', ' ']
--------------------
['X', 'X', 'O']
['O', 'O', 'X']
['X', ' ', ' ']
--------------------
['X', 'X', 'O']
['O', 'O', 'X']
['X', 'O', ' ']
--------------------
['X', 'X', 'O']
['O', 'O', 'X']
['X', 'O', 'X']
--------------------


In [81]:
# When one side is in a sure-to-lose state, it moves randomly.  
# eg: 
# _ O _
# X _ _
# _ _ _

# 課題2  
connect-four

In [13]:
import copy

class ConnectFour:
    def __init__(self):
        # 7x6
        self.players = ['X', 'O']
        
    def is_terminal(self, state):
        if self.is_draw(state):
            return True
        elif self.is_win(state,0) or self.is_win(state,1):
            return True
        return False
        
    def is_draw(self, state):
        if self.is_win(state,0) or self.is_win(state,1):
            return False
        for i in range(7):
            if state[i][0]==' ':
                return False
        return True

    def is_win(self, state, player_id):
        li=[]
        for i in range(7):
            for j in range(6):
                if state[i][j]==self.players[player_id]:
                    li.append([i,j])
        for i in li:
            if [i[0]+1,i[1]] in li and [i[0]+2,i[1]] in li and [i[0]+3,i[1]] in li:
                return True
            if [i[0],i[1]+1] in li and [i[0],i[1]+2] in li and [i[0],i[1]+3] in li:
                return True
            if [i[0]+1,i[1]+1] in li and [i[0]+2,i[1]+2] in li and [i[0]+3,i[1]+3] in li:
                return True
            if [i[0]-1,i[1]+1] in li and [i[0]-2,i[1]+2] in li and [i[0]-3,i[1]+3] in li:
                return True
        return False
        
    def make_move(self, state, move, player_id):
        new_state = copy.deepcopy(state)
        for i in range(6):
            if state[move][i]!=' ':
                char = self.players[player_id]
                new_state[move][i-1] = char
                return new_state
    
    

    def show(self,state):
        for i in state:
            print(i)
        print('-------------------------------------')
    
    def NegaAlphaBeta(self,state, player_id, alpha_value, beta_value):
        if self.is_draw(state):
            return 0,[]
        elif self.is_win(state, player_id):
            return 1,[]
        elif self.is_win(state, 1 - player_id):
            return -1,[]
        
        best_value = -100
        for i in range(7):
            if state[i][0]==' ':
                next_state = self.make_move(state, i, player_id)
                next_player = 1-player_id
                val, move1= self.NegaAlphaBeta(next_state, next_player,-beta_value,-alpha_value)
                val=-val
                if val > best_value:
                    best_value = val
                    best_move = i
                    alpha_value = max(alpha_value, val)
                if val>=beta_value:
                    return val,move1
        return best_value,best_move
    
    def Score(self):
        return NegaAlphaBeta(self.state, self.first_player,-100,100)

In [17]:
game = ConnectFour()                           
root_state = [[' ',' ','O','X','X','X'],
              [' ',' ',' ',' ','X','O'],
              [' ',' ','O','X','X','X'],
              ['O','X','O','O','O','X'],
              [' ','O','O','X','X','X'],
              [' ',' ',' ',' ','X','O'],
              [' ','O','O','X','O','O']]                           

score = game.NegaAlphaBeta(root_state, 0, -100, 100)
print(score)

(-1, 0)
